# Chapter 09 실전 자동화 프로젝트

## 9.2 프로젝트 자동화 구현하기

In [ ]:
# Step 1. 모듈 불러오기
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, PatternFill, Font, Side, Border, numbers

In [ ]:
# Step 2. 서비스 이력 데이터 불러오기
# 각 시트를 df_src로 불러오고, 이를 하나의 딕셔너리 자료형으로 묶어 반환해 줌
# 모든 시트 내용 불러오기 : sheet_name = None
df_src = pd.read_excel("chapter09/서비스 이력.xlsx", sheet_name = None, engine="openpyxl")

In [ ]:
# concat() 함수를 사용하여 8개 시트 데이터를 하나의 df로 병합 : 총 392개 행
df = pd.concat(df_src, ignore_index = True)    # True/False : 기존 인덱스 무시/유지
display(df)

In [ ]:
# loc의 슬라이싱 값에 필터 내용을 넣어 필터링 수행
df_filtered = df.loc[df["고객사명"] == "타오롱글로벌"]
display(df_filtered) 

In [ ]:
#customer_list = df["고객사명"]    # 392건
customer_list = df["고객사명"].unique()
print("지난 한달간 서비스를 받은 고객 수 : {}".format(len(customer_list)))
print("고객리스트 : \n{}".format(customer_list))

In [ ]:
for customer in customer_list:
    print(customer)

In [ ]:
# 송장 테이블의 필드 제목 만들기 : 전역 변수로 선언
invoice_table_header_list = [ 		# 항목 열(알파벳), 항목명, 셀 너비 순서
    ["A", "고객사", 20], ["B", "서비스 일자", 12], ["C", "서비스 내역", 40],
    ["D", "서비스 기사", 12], ["E", "유상/무상", 10], ["F", "공급가액(원)", 15],
    ["G", "부가세(원)", 15], ["H", "합계(원)", 15]
]

In [ ]:
# Step 3. 청구서 생성 함수 작성하기
def makeInvoiceWorkbook():
    wb = Workbook() 				# 워크북 객체 생성
    ws = wb["Sheet"] 				# 자동으로 생성된 첫 번째 시트 선택
    ws.title = "서비스 청구서"	# 선택된 첫 번째 시트의 이름 변경
    
    ws["A2"] = "(주)카피맨 렌탈 복사기 서비스 비용 청구서"
    ws["A4"] = "※서비스 기간 : 2021년 10월"
    
    # 청구서 표의 헤더 입력
    for header in invoice_table_header_list:
        ws[header[0] + "5"] = header[1]

    ws["A8"] = "위 금액을 청구합니다."
    ws["A9"] = "2021년 10월 29일"
    ws["A10"] = "(주)카피맨 대표이사 김카피(인)"
    
    return wb

In [ ]:
# 테스트용 엑셀 파일 저장해 보기
wb = makeInvoiceWorkbook()
wb.save("chapter09/서비스 청구서_테스트.xlsx")

In [ ]:
# 고객 리스트 별로 분류해서 108개의 엑셀 파일들 저장하기
for customer in customer_list:
    wb = makeInvoiceWorkbook()
    wb.save("chapter09/서비스 청구서_{}.xlsx".format(customer))

In [ ]:
# Step 4. 청구서 내용 입력하기
for customer in customer_list:
    wb = makeInvoiceWorkbook()				# 함수 call
    ws = wb["서비스 청구서"]
    
    # 반복문에서 현재 접근하고 있는 고객사 이름으로 필터링
    this_customer_items = df.loc[df["고객사명"] == customer] 
    
    # 방문일자, 기사명 순으로 정렬될 수 있도록 함
    this_customer_items = this_customer_items.sort_values(by = ["방문일자", "기사"])
    display(this_customer_items)	# 필터링 및 재정렬된 내용 확인
    
    break		# 테스트로 고객사 하나의 df만 확인하기 위해 반복문을 중지

In [ ]:
# strp 6 에서 복사해서 사용할 것
for customer in customer_list:
    wb = makeInvoiceWorkbook()
    ws = wb["서비스 청구서"]
    this_customer_items = df.loc[df["고객사명"] == customer] 
    this_customer_items = this_customer_items.sort_values(by = ["방문일자", "기사"])
    item_row_no = 6 #서비스 이력 내용이 입력되는 행의 번호
    for idx in range(len(this_customer_items)): #서비스 이력 개수만큼 반복
        ws.insert_rows(item_row_no) #서비스 이력이 들어갈 곳에 새로운 행 삽입
        # 고객사, 서비스 일자, 서비스 내역 등 데이터를 항목에 맞게 입력
        ws["A{}".format(item_row_no)] = this_customer_items.iloc[idx, 1]		#서비스 일자
        ws["B{}".format(item_row_no)] = this_customer_items.iloc[idx, 0]		#고객사
        ws["C{}".format(item_row_no)] = this_customer_items.iloc[idx, 3]		#서비스 기사
        ws["D{}".format(item_row_no)] = this_customer_items.iloc[idx, 2]		#서비스 내역
        ws["E{}".format(item_row_no)] = this_customer_items.iloc[idx, 4]		#유상/무상
        ws["F{}".format(item_row_no)] = this_customer_items.iloc[idx, 5]		#공급가액(원)
        ws["G{}".format(item_row_no)] = this_customer_items.iloc[idx, 6]		#부가세(원)
        ws["H{}".format(item_row_no)] = this_customer_items.iloc[idx, 7]		#합계(원)
        item_row_no += 1 # 다음 행에 서비스 이력이 입력될 수 있도록 변수값을 1 증가시킴
    ws["A{}".format(item_row_no)] = "총계"
    ws["B{}".format(item_row_no)] = '=COUNTA(C{}:C{})&"건"'.format(6, item_row_no-1)
    ws["F{}".format(item_row_no)] = "=SUM(F{}:F{})".format(6, item_row_no-1)
    ws["G{}".format(item_row_no)] = "=SUM(G{}:G{})".format(6, item_row_no-1)
    ws["H{}".format(item_row_no)] = "=SUM(H{}:H{})".format(6, item_row_no-1)

    wb.save("chapter09/서비스 청구서_{}.xlsx".format(customer)) # 결과 확인을 위해 엑셀 파일로 저장

In [ ]:
# Step 5. 청구서 서식 변경 함수 작성하기
# 워크시트와 서비스 이력이 들어간 영역의 시작 행과 종료 행을 파라미터로 받도록 설정함
def stylizeInvoiceForm(ws, item_start_row_no, item_row_no):
    # 서비스 이력표 테두리 서식과 적용 영역을 미리 지정
    side = Side(color = "CCCCCC", border_style = "medium")				# 밝은 회색
    border_style = Border(left = side, right = side, top = side, bottom = side)

    # ① 서비스 이력표 영역 정렬 및 폰트 서식 설정
    for header in invoice_table_header_list:
        ws.column_dimensions[header[0]].width = header[2]				# 셀 너비 설정
        
        # 헤더 행 서식 설정
        ws[header[0] + "5"].font = Font(sz="12", bold = True)		#"A5“~”H5” 셀의 폰트(12, 굵게)
        ws[header[0] + "5"].fill = PatternFill(patternType="solid", start_color="ffd663")
        ws[header[0] + "5"].border = border_style
        ws[header[0] + "5"].alignment = Alignment(horizontal = "center", vertical = "center")
        
        # 내용 행 서식 설정, 내용 시작 행부터 종료 행까지 반복문으로 접근하여 설정
        for row_no in range(item_start_row_no, item_row_no+1):
            ws["{}{}".format(header[0], row_no)].font = Font(sz = "11", bold = False)
            ws["{}{}".format(header[0], row_no)].border = border_style
            
            if header[0] in ["A", "B", "C", "D", "E"]: 			# A~E 열은 중앙 정렬
                ws["{}{}".format(header[0], row_no)].alignment = \
                    Alignment(horizontal = "center", vertical = "center")
            
            else: 				# F~H 열은 숫자(금액)이므로 오른쪽 정렬, 세 자릿수마다 콤마 입력
                ws["{}{}".format(header[0], row_no)].alignment = \
                    Alignment(horizontal = "right", vertical = "center")
                ws["{}{}".format(header[0], row_no)].number_format = \
                    numbers.BUILTIN_FORMATS[37]

        # 총계 행 서식 설정
        ws["{}{}".format(header[0], item_row_no)].font = Font(bold = True)
        ws["{}{}".format(header[0], item_row_no)].fill = \
            PatternFill(patternType = "solid", start_color = "eeeeee") 

    # ② 셀 병합 처리
    ws.merge_cells("A2:H2")				# (주)카피맨 렌탈 복사기 서비스 비용 청구서
    ws.merge_cells("A4:H4")				# ※서비스 기간…
    ws.merge_cells("B{}:E{}".format(item_row_no, item_row_no))			# 0건 cell
    ws.merge_cells("A{}:H{}".format(item_row_no+2, item_row_no+2))		# 위 금액을 …
    ws.merge_cells("A{}:H{}".format(item_row_no+3, item_row_no+3))		# 년 월 일
    ws.merge_cells("A{}:H{}".format(item_row_no+4, item_row_no+4))		# ㈜카피맨…


    # ③ 서비스 이력표 제외한 부분의 정렬/폰트 서식 설정
    ws["A2"].alignment = Alignment(horizontal = "center", vertical = "center")
    ws["A2"].font = Font(sz = "20", bold = True)
    ws["A4"].alignment = Alignment(horizontal = "right", vertical = "center")
    ws["A{}".format(item_row_no+2)].alignment = \
        Alignment(horizontal = "center", vertical = "center")
    ws["A{}".format(item_row_no+2)].font = Font(sz = "16", bold = True)
    ws["A{}".format(item_row_no+3)].alignment = \
        Alignment(horizontal = "center", vertical = "center")
    ws["A{}".format(item_row_no+3)].font = Font(sz = "12", bold = True)
    ws["A{}".format(item_row_no+4)].alignment = \
        Alignment(horizontal = "right", vertical = "center")
    ws["A{}".format(item_row_no+4)].font = Font(sz = "14", bold = True)

    # ④ 행별 높이 설정
    ws.row_dimensions[2].height = 40
    ws.row_dimensions[4].height = 20
    ws.row_dimensions[5].height = 25
    ws.row_dimensions[item_row_no+2].height = 40
    ws.row_dimensions[item_row_no+3].height = 20
    ws.row_dimensions[item_row_no+4].height = 40

    for row_no in range(item_start_row_no, item_row_no+1):
        ws.row_dimensions[row_no].height = 25
    
    #서식 처리가 완료된 워크시트 리턴
    return ws

In [ ]:
# Step 6. 서식 적용 후 엑셀 파일로 저장하기
for customer in customer_list:
    wb = makeInvoiceWorkbook()
    ws = wb["서비스 청구서"]
    this_customer_items = df.loc[df["고객사명"] == customer] 
    this_customer_items = this_customer_items.sort_values(by = ["방문일자", "기사"])
    item_row_no = 6 
    for idx in range(len(this_customer_items)): 
        ws.insert_rows(item_row_no)
        ws["A{}".format(item_row_no)] = this_customer_items.iloc[idx, 1]
        ws["B{}".format(item_row_no)] = this_customer_items.iloc[idx, 0]
        ws["C{}".format(item_row_no)] = this_customer_items.iloc[idx, 3]
        ws["D{}".format(item_row_no)] = this_customer_items.iloc[idx, 2]
        ws["E{}".format(item_row_no)] = this_customer_items.iloc[idx, 4]
        ws["F{}".format(item_row_no)] = this_customer_items.iloc[idx, 5]
        ws["G{}".format(item_row_no)] = this_customer_items.iloc[idx, 6]
        ws["H{}".format(item_row_no)] = this_customer_items.iloc[idx, 7]
        item_row_no += 1 
    ws["A{}".format(item_row_no)] = "총계"
    ws["B{}".format(item_row_no)] = '=COUNTA(C{}:C{})&"건"'.format(6, item_row_no-1)
    ws["F{}".format(item_row_no)] = "=SUM(F{}:F{})".format(6, item_row_no-1)
    ws["G{}".format(item_row_no)] = "=SUM(G{}:G{})".format(6, item_row_no-1)
    ws["H{}".format(item_row_no)] = "=SUM(H{}:H{})".format(6, item_row_no-1)
    
    # 아래 부분 추가하기
    ws = stylizeInvoiceForm(ws, 6, item_row_no)  # 청구서 서식 변경 함수 호출
    
    wb.save("chapter09/서비스 청구서_{}.xlsx".format(customer)) # 엑셀 파일로 저장